In [1]:
import pyspark
import os
os.environ['PYSPARK_PYTHON'] = '/usr/local/anaconda3/envs/spark/bin/python3'

In [2]:
sc = pyspark.SparkContext(appName="Matrix Multiplication")

In [3]:
matrixA = sc.textFile("matrixA.txt")

In [4]:
matrixA.collect()

['0 0 1', '0 1 2', '0 2 3', '1 0 4', '1 1 5', '1 2 6']

In [5]:
matrixB = sc.textFile("matrixB.txt")

In [6]:
def toPairsMatrixA(x):
    i, j, v = x.split()
    return (j, (i, v))

def toPairsMatrixB(x):
    j, k, v = x.split()
    return (j, (k, v))

In [7]:
pairMatrixA = matrixA.map(toPairsMatrixA)

In [8]:
pairMatrixB = matrixB.map(toPairsMatrixB)

In [9]:
pairMatrixA.collect()

[('0', ('0', '1')),
 ('1', ('0', '2')),
 ('2', ('0', '3')),
 ('0', ('1', '4')),
 ('1', ('1', '5')),
 ('2', ('1', '6'))]

In [22]:
grouped = pairMatrixA.join(pairMatrixB)

In [23]:
grouped.collect()

[('0', (('0', '1'), ('0', '6'))),
 ('0', (('0', '1'), ('1', '3'))),
 ('0', (('1', '4'), ('0', '6'))),
 ('0', (('1', '4'), ('1', '3'))),
 ('1', (('0', '2'), ('0', '5'))),
 ('1', (('0', '2'), ('1', '2'))),
 ('1', (('1', '5'), ('0', '5'))),
 ('1', (('1', '5'), ('1', '2'))),
 ('2', (('0', '3'), ('0', '4'))),
 ('2', (('0', '3'), ('1', '1'))),
 ('2', (('1', '6'), ('0', '4'))),
 ('2', (('1', '6'), ('1', '1')))]

In [28]:
def flatMap(x):
    list_a = x[1][0]
    list_b = x[1][1]
    return [((a[0], b[0]), int(a[1]) * int(b[1])) for a in list_a for b in list_b]

In [29]:
beforeAdd = grouped.flatMap(flatMap)

In [30]:
beforeAdd.collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 17.0 failed 1 times, most recent failure: Lost task 2.0 in stage 17.0 (TID 50, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/anaconda3/envs/spark/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in main
    process()
  File "/usr/local/anaconda3/envs/spark/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 367, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/anaconda3/envs/spark/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 390, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/anaconda3/envs/spark/lib/python3.7/site-packages/p